In [1]:
!pip install --user nltk

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd ./drive/MyDrive/git_hub_repo/NLP_tasks/NLP/Homework_2/ngram_authorship_train/


/content/drive/MyDrive/git_hub_repo/NLP_tasks/NLP/Homework_2/ngram_authorship_train


In [35]:
import nltk
from nltk.lm.preprocessing import padded_everygram_pipeline, pad_both_ends, flatten
from nltk.lm import MLE
from nltk.util import bigrams, trigrams, everygrams
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.lm import MLE, Laplace
from nltk.lm.models import StupidBackoff, WittenBellInterpolated

nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
f = open('austen_utf8.txt', 'r')
text_austen = f.read()

f = open('dickens_utf8.txt', 'r')
text_dickens = f.read()

f = open('tolstoy_utf8.txt', 'r')
text_tolstoy = f.read()

f =open('wilde_utf8.txt', 'r')
text_wilde = f.read()

In [20]:
import re
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)

    return text

In [21]:
def preprocess_text(text):
  sents = sent_tokenize(text, language='english')

  clean_sents = []
  for sent in sents:
    clean_sents.append(clean_text(sent))

  clean_words = []
  for sent in clean_sents:
    clean_words.append(word_tokenize(sent))

  return clean_words

In [22]:
def split_train_test(clean_words):
  train_size = len(clean_words) * 0.9
  train_sentences = clean_words[:int(train_size)]
  test_sentences = clean_words[int(train_size):]
  return train_sentences, test_sentences

In [23]:
def train_lm(train_words, n, method):
  train, vocab = padded_everygram_pipeline(n, train_words)

  if method == 'Laplace':

    lm = Laplace(n)

  elif method == 'MLE':

    lm = MLE(n)

  elif method == 'StupidBackoff':

    lm = StupidBackoff(order = n)

  elif method == 'WittenBellInterpolated':

    lm = WittenBellInterpolated(order = n)

  lm.fit(train, vocab)

  return lm

In [36]:
def test_lm(lm, test_words):
  test_paddedline  = list(everygrams(list(pad_both_ends(test_words, n=3)), max_len=3))
  return lm.perplexity(test_paddedline)

In [25]:
def show_accuracy_lm(author):

  if author == 'austen':
    test_set = austen_test

  elif author == 'dickens':
    test_set = dickens_test

  elif author == 'tolstoy':
    test_set = tolstoy_test

  elif author == 'wilde':
    test_set = wilde_test


  true_or_false_list = []

  for test_sent in test_set:

    austen_perplexity = test_lm(austen_lm, test_sent)
    dickens_perplexcity = test_lm(dickens_lm, test_sent)
    tolstoy_perplexity = test_lm(tolstoy_lm, test_sent)
    wilde_perplexity = test_lm(wilde_lm, test_sent)

    compares = {'austen': austen_perplexity,
                'dickens': dickens_perplexcity,
                'tolstoy': tolstoy_perplexity,
                'wilde': wilde_perplexity}

    min_perplexity = min(compares, key=compares.get)


    if author == min_perplexity:

      true_or_false_list.append(1)

    else:
      true_or_false_list.append(0)


  return (sum(true_or_false_list)/len(true_or_false_list))*100

In [37]:
austen_clean_words = preprocess_text(text_austen)
dickens_clean_words = preprocess_text(text_dickens)
tolstoy_clean_words = preprocess_text(text_tolstoy)
wilde_clean_words = preprocess_text(text_wilde)

print(austen_clean_words[8])

n=3

austen_train, austen_test = split_train_test(austen_clean_words)
dickens_train, dickens_test = split_train_test(dickens_clean_words)
tolstoy_train, tolstoy_test = split_train_test(tolstoy_clean_words)
wilde_train, wilde_test = split_train_test(wilde_clean_words)



austen_lm = train_lm(austen_train, n, method='Laplace')
dickens_lm = train_lm(dickens_train, n, method='Laplace')
tolstoy_lm = train_lm(tolstoy_train, n, method='Laplace')
wilde_lm = train_lm(wilde_train, n, method='Laplace')

austen_rate = show_accuracy_lm('austen')
dickens_rate = show_accuracy_lm('dickens')
tolstoy_rate = show_accuracy_lm('tolstoy')
wilde_rate = show_accuracy_lm('wilde')

print(f'austen_accuracy: {austen_rate}%')
print(f'dickens_accuracy: {dickens_rate}%')
print(f'tolstoy_accuracy: {tolstoy_rate}%')
print(f'wilde_accuracy: {wilde_rate}%')


# austen_lm = train_lm(austen_train, n, method='MLE')
# dickens_lm = train_lm(dickens_train, n, method='MLE')
# tolstoy_lm = train_lm(tolstoy_train, n, method='MLE')
# wilde_lm = train_lm(wilde_train, n, method='MLE')

# austen_rate = show_accuracy_lm('austen')
# dickens_rate = show_accuracy_lm('dickens')
# tolstoy_rate = show_accuracy_lm('tolstoy')
# wilde_rate = show_accuracy_lm('wilde')

# print(f'austen_accuracy: {austen_rate}%')
# print(f'dickens_accuracy: {dickens_rate}%')
# print(f'tolstoy_accuracy: {tolstoy_rate}%')
# print(f'wilde_accuracy: {wilde_rate}%')


# austen_lm = train_lm(austen_train, n, method='StupidBackoff')
# dickens_lm = train_lm(dickens_train, n, method='StupidBackoff')
# tolstoy_lm = train_lm(tolstoy_train, n, method='StupidBackoff')
# wilde_lm = train_lm(wilde_train, n, method='StupidBackoff')


# austen_rate = show_accuracy_lm('austen')
# dickens_rate = show_accuracy_lm('dickens')
# tolstoy_rate = show_accuracy_lm('tolstoy')
# wilde_rate = show_accuracy_lm('wilde')

# print(f'austen_accuracy: {austen_rate}%')
# print(f'dickens_accuracy: {dickens_rate}%')
# print(f'tolstoy_accuracy: {tolstoy_rate}%')
# print(f'wilde_accuracy: {wilde_rate}%')


# austen_lm = train_lm(austen_train, n, method='WittenBellInterpolated')
# dickens_lm = train_lm(dickens_train, n, method='WittenBellInterpolated')
# tolstoy_lm = train_lm(tolstoy_train, n, method='WittenBellInterpolated')
# wilde_lm = train_lm(wilde_train, n, method='WittenBellInterpolated')


# austen_rate = show_accuracy_lm('austen')
# dickens_rate = show_accuracy_lm('dickens')
# tolstoy_rate = show_accuracy_lm('tolstoy')
# wilde_rate = show_accuracy_lm('wilde')

# print(f'austen_accuracy: {austen_rate}%')
# print(f'dickens_accuracy: {dickens_rate}%')
# print(f'tolstoy_accuracy: {tolstoy_rate}%')
# print(f'wilde_accuracy: {wilde_rate}%')



['such', 'language', 'for', 'a', 'young', 'man', 'entirely', 'dependent', 'touse—nobody', 'but', 'you', 'mr', 'knightley', 'would', 'imagine', 'it', 'possible']
austen_accuracy: 79.20997920997921%
dickens_accuracy: 63.92785571142284%
tolstoy_accuracy: 40.40767386091127%
wilde_accuracy: 77.13248638838476%


In [27]:
prompt = [['you', 'are'], ['I', 'love'], ['The', 'weather'], ['Beautiful', 'world'], ['kindness', 'of']]

In [28]:
for i in prompt:

  austen_generated = austen_lm.generate(6, text_seed=i)
  dickens_generated = dickens_lm.generate(6, text_seed=i)
  tolstoy_generated = tolstoy_lm.generate(6, text_seed=i)
  wilde_generated = wilde_lm.generate(6, text_seed=i)

  print('austen', i, austen_generated, round(test_lm(austen_lm, i+austen_generated), 2))
  print('dickens', i, dickens_generated, round(test_lm(dickens_lm, i+dickens_generated), 2))
  print( 'tolstoy', i, tolstoy_generated, round(test_lm(tolstoy_lm, i+tolstoy_generated), 2))
  print('wilde', i, wilde_generated, round(test_lm(wilde_lm, i+wilde_generated), 2))

  print()




austen ['you', 'are'] ['mistaken', '</s>', '</s>', '</s>', '</s>', '</s>'] 43.61
dickens ['you', 'are'] ['not', 'likely', 'to', 'be', 'cried', 'over'] 1355.7
tolstoy ['you', 'are'] ['quite', 'a', 'scandal', 'to', 'all', 'and'] 2885.43
wilde ['you', 'are'] ['just', 'like', 'that', 'sort', 'of', 'magentadressingwrapper'] 1535.1

austen ['I', 'love'] ['with', 'her', 'or', 'with', 'a', 'look'] 2303.53
dickens ['I', 'love'] ['”', 'what', 'indeed', 'with', 'his', 'legs'] 4611.5
tolstoy ['I', 'love'] ['of', 'these', 'people', 'hetreated', 'them', 'with'] 6687.68
wilde ['I', 'love'] ['your', 'theoriesabout', 'pleasure', '</s>', '</s>', '</s>'] 513.28

austen ['The', 'weather'] ['was', 'so', 'far', 'each', 'recital', 'confirmed'] 4797.0
dickens ['The', 'weather'] ['gentlemen', '”', 'said', 'charles', 'darnay', 'mysteries'] 4078.18
tolstoy ['The', 'weather'] ['for', 'haying', '</s>', '</s>', '</s>', '</s>'] 340.05
wilde ['The', 'weather'] ['”', 'he', 'said', 'leaving', 'the', 'theatrewith'] 3314

In [29]:
['a', 'b'] + ['c', 'd']

['a', 'b', 'c', 'd']